In [4]:
import sys
sys.path.append("../src")

from utils.utils import load_files_from_specific_folder, read_file, get_filename_without_extension
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
## Ground truth extracted from original textual description with claude
### PROMPT: Given a textual description, your task is to identify all activities and actors involved in the process. 
# For activities identify atomic activities, so tasks that cannot be split in any subtasks, while for actors, only return them if two or more actors are explicitly mentioned. 
# Keep in mind that the pronoun "YOU" must not be considered as an actor.
# Return tasks and actors in two separate python lists of strings called task and actors. 
# In case no tasks or actors are identified, return empty lists. 
# Do not include any additional or explanatory text. 
# Here's the textual description:

In [ ]:
process_data = {
    "E_j01": {
        "tasks": [
            "report job applications",
            "write job applications",
            "send job offers",
            "confirm receipt of job applications",
            "rate application",
            "negotiate job interview",
            "enter probation phase",
            "rate company",
            "rate applicant",
            "see reviews",
            "continue receiving job offers",
            "stop reporting"
        ],
        "actors": [
            "companies",
            "job applicants"
        ]
    },
    "E_j02": {
        "tasks": [
            "Fetch information about potential models",
            "Let parent select",
            "Collect relevant information", 
            "Notify Social Security",
            "Notify Company in time",
            "Gather information from companies",
            "Let parent decide about extension"
        ],
        "actors": []
    },
    "E_j03": {
        "tasks": [
            "gather information about work accidents",
            "gather information about almost work accidents",
            "inform employer of work accident",
            "inform employer of incident that would almost have led to accident",
            "inform employer of serious and immediate risk to safety and health discovered",
            "inform employer of defect discovered in protection systems",
            "report work accident to Labour Inspectorate",
            "report work accident to emergency services",
            "report work accident to accident insurance provider",
            "report accidents involving employed people to accident insurance provider",
            "report accidents involving self-employed person to accident insurance provider",
            "report accidents to competent directorate",
            "report work accident to accident insurance provider in triplicate",
            "report accident in writing to private insurance provider",
            "report fatalities to private insurance provider"
        ],
        "actors": [
            "employees",
            "employer",
            "self-employed person",
            "students",
            "schoolchildren",
            "schools",
            "educational facilities",
            "universities",
            "insured party"
        ]
    },
    "E_j04": {
        "tasks": [
            "enter topic that interests you",
            "enter how much money you want",
            "enter experience",
            "enter hobbies",
            "accept offers",
            "deny offers",
            "write status updates about internship",
            "write status updates about intern",
            "recommend company to friends",
            "send tweets"
        ],
        "actors": []
    },
    "E_j05": {
        "tasks": [
            "check requirements for working and living in austria",
            "organize accommodation",
            "organize bank account",
            "negotiate accommodation",
            "negotiate bank account",
            "apply for visa",
            "renew visa every x amount of months",
            "apply for Rot-Weiss-Rot Card",
            "complete visa application form",
            "obtain travel document valid in Austria",
            "obtain passport photo",
            "present travel health insurance policy",
            "provide proof of sufficient means of subsistence",
            "provide other evidence requested by authorities",
            "contact representation in advance for information",
            "issue visa",
            "determine main travel destination",
            "determine territorial competence for visa applications",
            "submit application to competent consulate",
            "provide justification for exceptional competence claim",
            "apply for type D visa at Austrian representation",
            "carry out official actions in relation to visas"
        ],
        "actors": [
            "representation offices",
            "border control posts",
            "state police headquarters",
            "representation office"
        ]
    },
    "G_g01": {
        "tasks": [
            "send invitations to 10 friends",
            "find out which games friends want to play",
            "appoint date for LAN party",
            "agree on date",
            "appoint another date",
            "find out if you have all games guests want to play",
            "download missing games",
            "find out how much beer friends will require",
            "buy missing beer",
            "enjoy LAN party"
        ],
        "actors": []
    },
    "G_g03" : {
        "tasks" : [
            "create online tool for creating characters",
            "set up account",
            "come up with character names",
            "check whether you have battle.net account",
            "enter account information",
            "click link in confirmation mail",
            "check if you have active WoW subscription",
            "select payment method",
            "enter credit card information",
            "enter IBAN and BIC numbers",
            "log into game",
            "select realm",
            "select race",
            "select class",
            "enter character names",
            "get confirmation",
            "get selfies of character",
            "get message when expansion is released"
        ],
        "actors" : [
        ]
    },
    "G_j01" : {
        "tasks" : [
            "find out which resources you want",
            "collect resources",
            "create tools",
            "gather resources",
            "create custom logic of dependencies",
            "start farming",
            "strike with random natural disasters",
            "set bot back",
            "reach milestones",
            "notify when milestones are reached",
            "update list of to be farmed materials",
            "finish bot",
            "brag to selected friends",
            "send materials to friends"
            ],

        "actors" : [
        "Players",
        "bot"
    ]},
    "M_g01" : {
        "tasks" : [
            "instruct artist to create project",
            "send sketches",
            "tell artist what to change",
            "choose plastic color",
            "check how much color you have left",
            "put color on shopping list",
            "order color",
            "turn printer on",
            "heat up bed",
            "heat up extruder",
            "send back status",
            "generate gcode file",
            "print model"
    ],
        "actors" : [
    ]},
    "M_g02" : {
        "tasks" : [
            "develop basic design",
            "order Lego brick sets",
            "give lego sets to group of children",
            "sort parts into containers",
            "build subcomponents individually",
            "reorder individual parts",
            "test subcomponents individually",
            "assemble subcomponents",
            "redesign subcomponents",
            "rebuild subcomponents"
        ],
        "actors" : [
        ]
    },
    "M_j01": {
        "tasks" : [
            "tell you the properties",
            "start ordering parts from online sources",
            "arrive",
            "do manual inspection of all parts",
            "assemble parts",
            "send updates to customer",
            "produce first saw",
            "send saw to customer",
            "like the product",
            "produce rest of order"
        ],
        "actors": [],
        },
    "M_j02": {
        "tasks" : [
            "decide between bars",
            "decide number of seats",
            "decide seat color",
            "decide amount of water in toilets",
            "receive specifications",
            "send requirements to different teams",
            "manufacture parts",
            "manufacture vodka bar",
            "manufacture whiskey bar",
            "receive individual bits and pieces",
            "assemble interior of plane",
            "send plane on test flight",
            "create test protocol",
            "send test protocol",
            "deliver plane to customer",
            "confirm delivery"
        ],

        "actors" : [
            "customer",
            "Russian team",
            "Irish team"
        ]
    },
    "M_j03": {
        "tasks" : [
            "query web shops",
            "find out how much they can deliver",
            "find out price",
            "find out delivery time",
            "create list of parts to order from each shop",
            "order from several shops",
            "use cheapest parts",
            "arrive in batches",
            "start building",
            "reorder parts",
            "reorder cheapest parts",
            "reorder fastest parts",
            "write mail to friends"
        ],

        "actors" : []
    },
    "M_k05" : {
        "tasks" : [
            "collect requirements",
            "send requirements to tree house architect",
            "send back draft",
            "refine draft with additional requirements",
            "create list of needed materials from plan",
            "order materials from online stores",
            "process order",
            "send messages to friends to build house",
            "build house",
            "send invitations for tree house party",
            "create list of people that attend party"
        ],

        "actors" : [
            "tree house architect",
            "friends",
            "online stores"
        ]},
    "R_g01" : {
        "tasks" : [
            "enter model and what is broken",
            "give list of materials",
            "give list of tools",
            "give ordering choices for each part",
            "contact friends to borrow/buy something",
            "buy rest of items",
            "receive instructions how to fix phone",
            "send review",
            "send video of fixing",
            "send phone to expert"
        ],

        "actors" : [
        ]},
    "R_j01" : {
        "tasks" : [
            "collect relevant information from machine",
            "calculate Overall Equipment Efficiency",
            "stop machine",
            "send email to engineer",
            "restart machine",
            "continue measuring",
            "disconnect machine",
            "stop process"
            ],

        "actors" : [
        ]},
    "R_j02" : {
        "tasks" : [
            "enter type of machine",
            "enter serial number",
            "begin inspection",
            "ask questions",
            "input values",
            "press buttons",
            "collect values automatically from machine",
            "show values",
            "ask additional questions"
        ],

        "actors" : [
            "application",
            "machine"
        ]},
    "R_j03" : {
        "tasks" : [   
            "describe symptoms",
            "send symptoms and fridge type to central fridge service center",
            "select local service facility",
            "make appointment",
            "arrive at random time",
            "leave without repairing fridge",
            "repair fridge",
            "confirm fridge is OK again",
            "rate service facility"
        ],
        "actors" : [
            "central fridge service center",
            "service facility"
        ]
    },
    "R_j04" : {
        "tasks" : [
            "enter type of machine",
            "enter serial number",
            "begin inspection",
            "ask questions",
            "input values",
            "press buttons",
            "collect values automatically from machine",
            "show values",
            "ask additional questions"
        ],
        "actors" : [
            "application",
            "machine"
        ]
    },
    "V_g01" :{ 
        "tasks" : [
            "receive order",
            "ask whether customer wants menu or just burger",
            "want menu or just burger",
            "start preparing drink",
            "ask if you want fries or wedges",
            "want fries or wedges",
            "prepare fries",
            "prepare wedges",
            "prepare burger",
            "give enthusiastic status updates every 30 seconds",
            "deliver order via conveyor belt"
        ],

        "actors" : [
            "robot",
            "customer",
            "conveyor belt"
        ]},
    "V_k08" : {
        "tasks" : [
            "log into E-Shop system",
            "wait for login confirmation",
            "check credentials",
            "select products",
            "add product to shopping cart",
            "save product",
            "check if every product was selected",
            "finish order",
            "prepare payment and shipment",
            "enter payment data",
            "wait until bank confirms payment",
            "confirm payment",
            "enter shipping address",
            "enter billing address",
            "finish order by system"
        ],
        "actors" : [
            "customer",
            "system",
            "bank"
        ]},
    "V_k09": {
        "tasks" : [
            "read order from automatic order management system",
            "check if first product from order is in stock",
            "withdraw from warehouse",
            "reorder from wholesaler",
            "wait for arrival of product",
            "demand delivery delay penalty from wholesaler",
            "arrive",
            "register in stock management system",
            "check if whole order is ready for shipment",
            "request courier",
            "pack products",
            "ship products",
            "select next product from order"
        ],

        "actors" : []
    },
    "X_g01" : {
        "tasks" : [
            "select course",
            "tell you courses and dates",
            "select one",
            "check if you already have account at sports institute",
            "check whether you are registered at eligible university",
            "register your account",
            "request activation",
            "wait for response",
            "log into account",
            "connect to twitter account",
            "let you tweet to friends",
            "complete registration for course",
            "provide payment information",
            "receive course ticket"
        ],
        "actors" : [
            "system",
            "application"
        ]},
    "X_g03": {
        "tasks" : [
            "select starting block",
            "run for 5km",
            "measure time",
            "train",
            "check until goal achieved",
            "get starting number",
            "find out whether you can go from home or work",
            "go from home",
            "leave directly from work",
            "run at Night Run",
            "drink while running",
            "receive final running time",
            "enter information into app",
            "collect information"
        ],
        "actors" : [
        ]},                   
}

In [ ]:
def extract_bpmn_names(bpmn_dict):
    """
    Extracts lane names and task names from a BPMN dictionary structure.
    
    Parameters:
    bpmn_dict (dict): A dictionary containing the BPMN process definition
    
    Returns:
    tuple: (lane_names, task_names) where both are lists of strings
    """
    # Extract lane names
    lane_names = []
    if 'lanes' in bpmn_dict:
        lane_names = [lane['name'] for lane in bpmn_dict['lanes']]
    
    # Extract task names from all nodes
    supported_task = ['task', 'userTask', 'serviceTask', 'sendTask', 'receiveTask', 'manualTask', 'businessRuleTask', 'scriptTask']
    task_names = []
    task_types = []
    gateways = [ ]
    if 'nodes' in bpmn_dict:
        for node in bpmn_dict['nodes']:
            if node['BPMNtype'] in supported_task:
                task_names.append(node['name'])
                task_types.append(node['BPMNtype'])
            elif node['BPMNtype'] == 'exclusiveGateway' or node['BPMNtype'] == 'parallelGateway':
                gateways.append(node['BPMNtype'])
    
    return lane_names, task_names, task_types, gateways


In [ ]:
# files = load_files_from_specific_folder("../data/textual_aderenche", ".bpmn")    ## ADJUST THIS PATH AS NEEDED
# cretae empy dictionaries to store results
results = {}

for f in files:
    file_name = get_filename_without_extension(f)
    
    file = read_file(f)
    lane_names, task_names, _, _ = extract_bpmn_names(ast.literal_eval(file))
    
    # Initialize and store
    results[file_name] = {
        "task_names": task_names,
        "lane_names": lane_names
    }

In [ ]:
def component_similarity_metric(ground_truth, predictions):
    """
    Calculate similarity between lists of actor names using sentence transformer embeddings,
    cosine similarity, and F1 score.
    
    Parameters:
    ground_truth (list): List of ground truth actor names
    predictions (list): List of predicted actor names
    
    Returns:
    dict: Dictionary containing matching pairs and evaluation metrics
    """
    # If either list is empty, handle appropriately
    if len(ground_truth) == 0 and len(predictions) == 0:
        f1=1
        return f1
    
    if len(ground_truth) == 0:
        f1=0
        return f1
    
    if len(predictions) == 0:
        f1=0
        return f1
    
    # Load sentence transformer model
    #model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', token=False)  # Smaller, faster model
    # For better accuracy but slower performance:
    #model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', token=False)  # Larger model for better accuracy 
    model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1', token=False)  # Balanced model
    
    # Generate embeddings
    gt_embeddings = model.encode(ground_truth)
    pred_embeddings = model.encode(predictions)
    
    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(pred_embeddings, gt_embeddings)
    
    # Match each prediction to the most similar ground truth name
    matched_pairs = []
    used_gt_indices = set()
    
    # Sort predictions by their maximum similarity score (match most confident first)
    pred_max_scores = np.max(similarity_matrix, axis=1)
    sorted_pred_indices = np.argsort(-pred_max_scores)
    
    for pred_idx in sorted_pred_indices:
        # Get best matching ground truth index
        gt_similarities = similarity_matrix[pred_idx]
        
        # Sort ground truth indices by similarity
        sorted_gt_indices = np.argsort(-gt_similarities)
        
        # Find best available match
        best_gt_idx = None
        for gt_idx in sorted_gt_indices:
            if gt_idx not in used_gt_indices:
                similarity_score = gt_similarities[gt_idx]
                # Only match if similarity is above threshold
                if similarity_score >= 0.7:  # Threshold for semantic similarity
                    best_gt_idx = gt_idx
                    used_gt_indices.add(gt_idx)
                    break
        
        if best_gt_idx is not None:
            matched_pairs.append({
                "pred": predictions[pred_idx],
                "gt": ground_truth[best_gt_idx],
                "similarity": float(gt_similarities[best_gt_idx])
            })
    
    # Calculate metrics
    true_positives = len(matched_pairs)
    false_positives = len(predictions) - true_positives
    false_negatives = len(ground_truth) - true_positives
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1

In [ ]:
import numpy as np
from pprint import pprint   # optional, just for nicer printing

# --- rename for brevity -------------------------------------------------------
predictions = results  # <-- the big dict with 'task_names'
ground_truth = process_data           # <-- the dict that holds the reference list

# --- collect one F1 per task file --------------------------------------------------
f1_scores_task = {}

for pid, gt in ground_truth.items():
    if pid not in predictions:                 # safety check
        print(f"[WARN] No predictions for {pid}; skipped.")
        continue

    gt_tasks   = [t.lower().strip() for t in gt["tasks"]]
    pred_tasks = [t.lower().strip() for t in predictions[pid]["task_names"]]

    f1_task     = component_similarity_metric(gt_tasks, pred_tasks)
    f1_scores_task[pid] = f1_task
    
# --- collect one F1 per actor file --------------------------------------------------
# (This part is similar to the previous one, but for actors)

f1_scores_actors ={}
for pid, gt in ground_truth.items():
    if pid not in predictions:                 # safety check
        print(f"[WARN] No predictions for {pid}; skipped.")
        continue

    #gt_actors   = [t.lower().strip() for t in gt["actors"]]
    #pred_actors = [t.lower().strip() for t in predictions[pid]["lane_names"]]

    f1_actor = component_similarity_metric(gt["actors"], predictions[pid]["lane_names"])
    f1_scores_actors[pid] = f1_actor
    
# sum up all F1 score for tasks and actors and take the mean, but do it weighted like 0.7 for task and 0.3 for actors
f1_scores = {}
for pid in f1_scores_task.keys():
    if pid in f1_scores_actors:
        f1_scores[pid] = 0.8 * f1_scores_task[pid] + 0.2 * f1_scores_actors[pid]
    else:
        f1_scores[pid] = f1_scores_task[pid]
        
# retunr overall mean
overall_f1 = np.mean(list(f1_scores.values()))
# and standard deviation
overall_std = np.std(list(f1_scores.values()))
print(overall_f1, overall_std)
    
